In [1]:
from pathlib import Path

import automech as am
import polars as pl
from automech.reaction import Reaction
from project_utilities import p_

open_browser = False
tag = "Z_mess_v0"
root_path = Path("../..")

stoich1 = "C5H7O2"
stoich2 = "C5H9O2"

mech1_json = p_.calculated_pes_mechanism(tag, stoich1, "json", path=p_.data(root_path))
mech2_json = p_.calculated_pes_mechanism(tag, stoich2, "json", path=p_.data(root_path))
mech1 = am.io.read(mech1_json)
mech2 = am.io.read(mech2_json)


mech1.reactions = mech1.reactions.filter(
    ~pl.col(Reaction.reactants).list.contains("O2(6)")
    & ~pl.col(Reaction.products).list.contains("O2(6)")
)

mech2.reactions = mech2.reactions.filter(
    ~pl.col(Reaction.reactants).list.contains("O2(6)")
    & ~pl.col(Reaction.products).list.contains("O2(6)")
)

if open_browser:
    disp_mech1 = mech1.model_copy(deep=True)
    disp_mech2 = mech2.model_copy(deep=True)
    disp_mech1.reactions = disp_mech1.reactions.filter(~pl.col("well_skipping"))
    disp_mech2.reactions = disp_mech2.reactions.filter(~pl.col("well_skipping"))
    am.display(disp_mech1, out_name=f"{stoich1}.html")
    am.display(disp_mech2, out_name=f"{stoich2}.html")

In [2]:
import altair as alt
import numpy as np
from autochem.util import plot
from project_utilities import fig

P_range = (0.1, 100)
T = 700
T_range = (500, 1200)
P = 10
total = False
legend = True
min_branch_frac = 0.00001
rate_df1a = fig.rate_data(
    mech1,
    reactants=("S(602)",),
    min_branch_frac=min_branch_frac,
    # colors=[plot.Color.green, plot.Color.purple, plot.Color.red],
)

rate_df1b = fig.rate_data(
    mech1,
    reactants=("S(1206)r0",),
    min_branch_frac=min_branch_frac,
    # colors=[plot.Color.green, plot.Color.purple, plot.Color.red],
)

rate_df2 = fig.rate_data(
    mech2,
    reactants=("S(719)",),
    min_branch_frac=min_branch_frac,
    # colors=[plot.Color.green, plot.Color.blue],
)

# Determine overall branching fraction range
_, f_data1a = fig.branching_fraction_xy_data(
    rate_df1a, T=T, P_range=P_range, total=True, units={"substance": "molecules"}
)
_, f_data1b = fig.branching_fraction_xy_data(
    rate_df1b, T=T, P_range=P_range, total=True, units={"substance": "molecules"}
)
_, f_data2 = fig.branching_fraction_xy_data(
    rate_df2, T=T, P_range=P_range, total=True, units={"substance": "molecules"}
)

f_min = np.nanmin([np.nanmin(f_data1a), np.nanmin(f_data1b), np.nanmin(f_data2)])
f_max = np.nanmax([np.nanmax(f_data1a), np.nanmax(f_data1b), np.nanmax(f_data2)])
f_range = (f_min, f_max)
print(f_range)

# Determine overall rate range
_, y_data1a = fig.rate_xy_data(
    rate_df1a, T_range=(500, 1200), P=P, total=total, units={"substance": "molecules"}
)
_, y_data1b = fig.rate_xy_data(
    rate_df1b, T_range=(500, 1200), P=P, total=total, units={"substance": "molecules"}
)
_, y_data2 = fig.rate_xy_data(
    rate_df2, T_range=(500, 1200), P=P, total=total, units={"substance": "molecules"}
)

y_min = np.nanmin([np.nanmin(y_data1a), np.nanmin(y_data1b), np.nanmin(y_data2)])
y_max = np.nanmax([np.nanmax(y_data1a), np.nanmax(y_data1b), np.nanmax(y_data2)])
y_range = (y_min, y_max)
print(y_range)

(np.float64(6.15259067922191e-06), np.float64(0.17375000000000002))
(np.float64(4.2244737879292553e-07), np.float64(53903979.07537629))


In [3]:
branch_frac_chart1a = fig.branching_fraction_chart(
    rate_df1a,
    P_range=(0.1, 100),
    T=T,
    legend=legend,
    total=True,
    y_range=f_range,
)
rate_chart1a = fig.rate_chart(
    rate_df1a,
    T_range=(500, 1200),
    P=P,
    legend=legend,
    total=total,
    y_range=y_range,
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart1a.properties(width=300, height=150),
    rate_chart1a.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [4]:
branch_frac_chart1b = fig.branching_fraction_chart(
    rate_df1b,
    P_range=(0.1, 100),
    T=T,
    legend=legend,
    total=True,
    y_range=f_range,
)
rate_chart1b = fig.rate_chart(
    rate_df1b,
    T_range=(500, 1200),
    P=P,
    legend=legend,
    total=total,
    y_range=y_range,
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart1b.properties(width=300, height=150),
    rate_chart1b.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [5]:
branch_frac_chart2 = fig.branching_fraction_chart(
    rate_df2,
    P_range=(0.1, 100),
    T=T,
    legend=legend,
    total=True,
    mark_kwargs={"strokeDash": [8, 10], "strokeCap": "round"},
    y_range=f_range,
)
rate_chart2 = fig.rate_chart(
    rate_df2,
    T_range=(500, 1200),
    P=P,
    legend=legend,
    total=total,
    y_range=y_range,
    mark_kwargs={"strokeDash": [8, 10], "strokeCap": "round"},
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart2.properties(width=300, height=150),
    rate_chart2.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [6]:
branch_frac_chart = alt.layer(branch_frac_chart1a, branch_frac_chart2).resolve_scale(
    color="independent"
)
rate_chart = alt.layer(rate_chart1a, rate_chart2).resolve_scale(color="independent")

chart = alt.vconcat(
    branch_frac_chart.properties(width=300, height=150),
    rate_chart.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)